# Genetic Algorithm

GA is a simple optimisation algorithm that mimics evolution. It starts with a random population of elements, and evaluates how <i>fit for survival</i> they are. Elements are then randomly selected in pairs to mate and produce a new generation of offspring. The selection ensures that elements with higher fitness are more likely to be chosen for mating, thus increasing the chances that their offspring retains the characteristics that made their parents fit in the first place. Random mutation can also appear.<br>
If an optimisation problem is too complex to be solved via gradient descent methods, GA can still give an answer.<br><br>
In this tutorial we are going to use GA to find a set of parameters that fit a signal.<br>
More details about the GA are given in the following sections.

In [ ]:
# IMPORTS
import math
import random
import numpy
import matplotlib.pyplot as plt

##### Definition of element

In this example, an element represents a composite wave signal $W(t) = \sum_{k=1}^n A_k \cos\left( 2\pi k t\right)$.<br><br>
Thus, the element can be represented by the list of amplitudes for each wave component: $\mathbf{e} = \{ A_1, A_2, ... A_n\}$.<br><br>
The training set is a collection of (t,y) points corresponding to the signal we are trying to fit.

In [ ]:
# set a decent population size to work with
populationSize = 100

# set the amount of plane waves
nWaves = 5

# create a random population
# each amplitude is a random number between -1 and 1
population = 2 * numpy.random.rand(populationSize, nWaves) - 1

# print the first 2 elements to see what we are dealing with!
print population[0:3]

# load a training set
trainT = numpy.load("./data/ga-train-T.npy") # time values
trainY = numpy.load("./data/ga-train-Y.npy") # corresponding signal values

# plot the training set
plt.plot(trainT, trainY, 'o')
plt.show()

##### Mixing elements

After the population is evaluated, we take pairs of elements at random, and mix them to produce an offspring population.
The simplest mixing function would be the one that assign each DNA component of the offspring from either parent at 
random.

In [ ]:
# define the mixing function
def Mix(father, mother):

    # create an empty child
    child = numpy.zeros(father.shape)
    
    # loop through the genes
    for c in xrange(len(child)):
        r = random.random() # random uniform number
        if r>0.5:
            child[c] = father[c] # take from father, OR ...
        else:
            child[c] = mother[c] # take from mother

    return child

##### Mutating elements

Occasionally, mutation can appear when mixing two elements. There are several ways to implement mutations, but here we stick to simplicity. A mutation adds a random value to one random gene of the element. <br>
Mutations should be rare or they will mess up the evolution!

In [ ]:
# define the mixing function
def Mutate(element):

    # random index of the gene to mutate
    idx = numpy.random.randint(0, nWaves)
    # random amount to add - between -1 and 1
    r = 2*numpy.random.rand()-1
    # apply the mutation
    element[idx] += r

    
# also define the probability of mutation occurring
mutationRate = 0.001 # -> on average, 0.1% of offspring will have a mutation


##### Evaluating fitness

This function takes in the element descriptor, performs the calculations necessary to estimate how well it performs, and returns its fitness.<br>
It is actually easy to calculate its badness (mean square error) and flip the sign. This way, the perfect element will have 0 fitness, while any mismatch with the training will give a negative value.

In [ ]:
def EvaluateFitness(element):

    # prepare a 0 array
    y = numpy.zeros(len(trainT))
    freq = 1 # starting frequency
    
    for amp in element: # loop through the amplitudes
        y += amp * numpy.cos(2*numpy.pi*freq*trainT) # add the wave
        freq += 1 # increment the frequency for the next wave
    
    # compute square error
    y = (y-trainY)*(y-trainY)
    
    return -numpy.mean(y)

##### Evolving...

Let the population evolve!

In [ ]:
# USE THIS TO RESET THE POPULATION
# DO NOT RUN IF NOT NEEDED!
population = 2 * numpy.random.rand(populationSize, nWaves) - 1

In [ ]:
# number of generations to compute
nGens = 30

stats = numpy.zeros((nGens,3))
best = None

# start the evolution loop!
for g in xrange(nGens):

    # evaluate all elements
    fitness = numpy.apply_along_axis(EvaluateFitness,1,population)
    # sort the elements by fitness
    order = numpy.argsort(-fitness)
    ordered = numpy.asarray([population[i] for i in order])

    # save some statistics and info
    stats[g,0] = -fitness[order[0]]
    stats[g,1] = -numpy.mean(fitness)
    stats[g,2] = -fitness[order[populationSize-1]]
    best = ordered[0]
    
    # select pairs at random
    indexes = numpy.random.triangular(0,0,1, populationSize*2)
    indexes = (numpy.floor((populationSize)*indexes)).astype(numpy.int)
    
    idx = 0
    for p in xrange(populationSize):

        p1 = ordered[indexes[idx]]; idx += 1
        p2 = ordered[indexes[idx]]; idx += 1
        # mix the two
        child = Mix(p1, p2)
        # mutate if needed
        r = numpy.random.rand()
        if r < mutationRate:
            Mutate(child)
        # save it in the population
        population[p] = child

In [ ]:
# evolution plot
plt.plot(stats[:,0],'o-', label='best')
plt.plot(stats[:,1],'o-', label='average')
plt.plot(stats[:,2],'o-', label='worst')
plt.legend()
plt.show()

In [ ]:
# best element info
print best
t = numpy.arange(0,1,0.01)
y = t * 0
freq = 1 # starting frequency

for amp in best: # loop through the amplitudes
    y += amp * numpy.cos(2*numpy.pi*freq*t) # add the wave
    freq += 1 # increment the frequency for the next wave
        
plt.plot(trainT, trainY, 'o', label='training points')
plt.plot(t, y, '-', label='latest model')
plt.legend()
plt.show()


##### Final remarks
Do not be fooled by the speed of this example... in real applications, training an AI with GA can take up to several weeks on supercomputers.<br>
You can see one in action <a href="www.nanolayers.com/copter/">here!</a> Try to beat it!